# Classificação de imagens de tomografia computadorizada de próstata

## Instalações

In [88]:
import sys

In [89]:
!{sys.executable} -m pip install pydicom

In [90]:
!{sys.executable} -m pip install pypng

In [91]:
!{sys.executable} -m pip install keras

In [92]:
!{sys.executable} -m pip install tensorflow

## Definição de paths e arquivos treino / teste

In [93]:
import pylab
import pydicom as dicom
import glob
import png
import os

# dcm to png

training_data = 'imagens/treinamento'
validation_data = 'imagens/validacao'

diretorio_list = ['imagens/treinamento/true/', 'imagens/validacao/true/', 'imagens/treinamento/false/', 'imagens/validacao/false/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.dcm'):
          mri_file = open(img, 'rb')
          imagem = dicom.read_file(mri_file)
          mri_file.close()

          shape = imagem.pixel_array.shape

          image_2d = []
          max_val = 0
          for row in imagem.pixel_array:
              pixels = []
              for col in row:
                  pixels.append(col)
                  if col > max_val: max_val = col
              image_2d.append(pixels)

          image_2d_scaled = []
          for row in image_2d:
              row_scaled = []
              for col in row:
                  col_scaled = int((float(col) / float(max_val)) * 255.0)
                  row_scaled.append(col_scaled)
              image_2d_scaled.append(row_scaled)

          png_nome = os.path.basename(img)[:-3] + 'png'
          print(png_nome)
          png_file = open(diretorio+png_nome, 'wb')
          w = png.Writer(shape[1], shape[0], greyscale=True)
          w.write(png_file, image_2d_scaled)
          png_file.close()


0000-12-000007.png
0002-12-000007.png
0005-12-000007.png
0014-12-000007.png
0015-12-000007.png
0018-12-000007.png
0019-12-000007.png
0026-12-000007.png
0028-12-000007.png
0031-12-000007.png
0035-12-000007.png
0037-12-000007.png
0040-12-000007.png
0041-12-000007.png
0046-12-000007.png
0077-12-000007.png
0078-12-000007.png
0082-12-000007.png
0092-12-000007.png
0094-12-000007.png
0095-12-000007.png
0097-12-000007.png
0099-12-000007.png
0102-12-000007.png
0103-12-000007.png
0104-12-000007.png
0105-12-000007.png
0106-12-000007.png
0111-12-000007.png
0114-12-000007.png
0115-12-000007.png
0117-12-000007.png
0118-12-000007.png
0119-12-000007.png
0121-12-000007.png
0123-12-000007.png
0125-12-000007.png
0126-12-000007.png
0127-12-000007.png
0128-12-000007.png
0129-12-000007.png
0131-12-000007.png
0139-12-000007.png
0143-12-000007.png
0054-12-000007.png
0055-12-000007.png
0067-12-000007.png
0071-12-000007.png
0164-12-000007.png
0169-12-000007.png
0179-12-000007.png
0181-12-000007.png
0183-12-0000

In [94]:
import keras
import numpy as np

import os
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

img_width, img_height = 256, 256

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        training_data,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')


Found 152 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [97]:
# Initialising the CNN
model = Sequential()

# Convolution
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolutional layer
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())


# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(256, 256,...)`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3))`


## Treinamento

In [98]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

nb_epoch  = 10
nb_train_samples = 120
nb_validation_samples = 64

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=7, epochs=10, validation_steps=64)`
  


Epoch 1/10
7/7 [==============================] - 13s 2s/step - loss: 1.7728 - acc: 0.6094 - val_loss: 0.5666 - val_acc: 0.8136
Epoch 2/10
7/7 [==============================] - 11s 2s/step - loss: 0.6040 - acc: 0.7320 - val_loss: 0.4871 - val_acc: 0.8136
Epoch 3/10
7/7 [==============================] - 12s 2s/step - loss: 0.6348 - acc: 0.6964 - val_loss: 0.5382 - val_acc: 0.8136
Epoch 4/10
7/7 [==============================] - 11s 2s/step - loss: 0.6148 - acc: 0.7398 - val_loss: 0.4953 - val_acc: 0.8136
Epoch 5/10
7/7 [==============================] - 12s 2s/step - loss: 0.6182 - acc: 0.7143 - val_loss: 0.5106 - val_acc: 0.8136
Epoch 6/10
7/7 [==============================] - 11s 2s/step - loss: 0.6518 - acc: 0.6953 - val_loss: 0.5632 - val_acc: 0.8136
Epoch 7/10
7/7 [==============================] - 11s 2s/step - loss: 0.6109 - acc: 0.7143 - val_loss: 0.5206 - val_acc: 0.8136
Epoch 8/10
7/7 [==============================] - 12s 2s/step - loss: 0.6057 - acc: 0.7308 - val_loss: 0

## Validação

In [99]:
from keras.preprocessing import image
from PIL import Image

diretorio_list = ['imagens/validacao/true/', 'imagens/validacao/false/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.png'):
        print(Image.open(img).filename[-18:-14])
        img = image.load_img(img, target_size=(256, 256))
        res = image.img_to_array(img)
        res = np.expand_dims(res, axis=0) * 1./255
        prediction = model.predict(res)
        
        print("Resultado: ", prediction);

0054
Resultado:  [[0.13393292]]
0055
Resultado:  [[0.15398242]]
0067
Resultado:  [[0.16973577]]
0071
Resultado:  [[0.16690482]]
0164
Resultado:  [[0.15193719]]
0169
Resultado:  [[0.14813894]]
0179
Resultado:  [[0.15367652]]
0181
Resultado:  [[0.14087255]]
0183
Resultado:  [[0.173893]]
0184
Resultado:  [[0.16642618]]
0186
Resultado:  [[0.16424978]]
0050
Resultado:  [[0.15002893]]
0051
Resultado:  [[0.13352564]]
0052
Resultado:  [[0.14525694]]
0053
Resultado:  [[0.14999063]]
0054
Resultado:  [[0.13393292]]
0056
Resultado:  [[0.19179662]]
0057
Resultado:  [[0.13807271]]
0058
Resultado:  [[0.16066304]]
0059
Resultado:  [[0.16895467]]
0060
Resultado:  [[0.12779313]]
0061
Resultado:  [[0.15581957]]
0062
Resultado:  [[0.15595497]]
0063
Resultado:  [[0.14833158]]
0064
Resultado:  [[0.12668408]]
0065
Resultado:  [[0.17401993]]
0073
Resultado:  [[0.16834594]]
0074
Resultado:  [[0.14567478]]
0075
Resultado:  [[0.12600376]]
0076
Resultado:  [[0.16273212]]
0079
Resultado:  [[0.166544]]
0080
Resulta